### Group Details

1. Group ID - 112
2. Dataset Name - Movie Review - (Data-Review_Polarity)
3. Assignment 1 - Problem 14

Members
1. PRAVIN KOSHTI (2020FC04406)
2. GHOSH SOHAM GYANPRAKASH TRIPTI (2020FC04408)
3. SHWETA PATEL (2020FC04414)


### Import required libraries and ignore the warning messages.

In [1]:
# If missing, then install the required libabries here.

# pip install nltk

In [2]:
import tarfile 
import nltk
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer       #lemmatization
from nltk.corpus.reader import NOUN, VERB, ADJ, ADV
from collections import defaultdict
from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
import string #punctuation
from nltk.corpus import stopwords
import os #read documents
import re #url

from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from random import shuffle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import GaussianNB             #Import Gaussian Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics                            #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestClassifier    #RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier #GradientBoostingClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Download stopwords and use English stopwords as we have reviews in english language.
nltk.download('stopwords')
stop_en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P10506243\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# sets of punctuation
punctuation_translator = str.maketrans("","",string.punctuation)

### Extract the data

In [5]:
## open file 
#import tarfile 
#file = tarfile.open('review_polarity.tar.gz') 
# extracting file 
#file.extractall('./Extracted Data') 
#file.close()

### Read and clean the data

Apply Tokenization (It will split the review data into single word).
Remove single character, punctuation, stop words and web links
Apply lemmitization (The process of converting a word to its base form)
Apply POS Tag (POS tagger is used to assign grammatical information of each word of the sentence.)

In [6]:
def read_document_and_remove_messy_text(path, stop_words, punctuation_translator):
    file = open(path, 'rb');
    content = file.read()
    words = word_tokenize(str(content))
    
    words_without_punctuation = []  
    for word in words:
        word = word.translate(punctuation_translator)
        if len(word)>2:  #don't add empty strings or irrelevant one
            words_without_punctuation.append(word)
    #print(words_without_punctuation)
    
    words_without_stop_words_and_punctuation = [word for word in words_without_punctuation if not word in stop_en]
    #print(words_without_stop_words_and_punctuation)
    
    words_without_stop_words_punctuation_and_url = []
    for word in words_without_stop_words_and_punctuation:
        word = re.sub(r"http\S+", "", word)
        words_without_stop_words_punctuation_and_url.append(word)
    
    # Lemmatization is the process of converting a word to its base form
    lemmatizer = WordNetLemmatizer()
    tag_map = defaultdict(lambda : wordnet.NOUN)
    tag_map['N'] = wordnet.NOUN
    tag_map['J'] = wordnet.ADJ
    tag_map['V'] = wordnet.VERB
    tag_map['R'] = wordnet.ADV
    
    # POS Tagging in NLTK is a process to mark up the words in text format for a particular part of a speech based on its definition and context
    tags = nltk.pos_tag(words_without_stop_words_punctuation_and_url)
    lemmas = [lemmatizer.lemmatize(token,tag_map[tag[0]]) for token,tag in tags]
    
    return lemmas


In [7]:
# Read each and every file, and pass the review data for cleansing purpose.

def clean_all_documents(folder_path, stop_words, punctuation_translator):
    file_names = os.listdir(folder_path)
    documents = [read_document_and_remove_messy_text(folder_path+'/'+file_name, stop_words, punctuation_translator) 
                 for file_name in file_names]
    return documents

negative_docs = clean_all_documents("Extracted Data/txt_sentoken/neg",stop_en, punctuation_translator)
positive_docs = clean_all_documents("Extracted Data/txt_sentoken/pos",stop_en, punctuation_translator)

In [8]:
# create list for all consolidated positive and negative reviews
all_documents_string_array = []
for i in range(len(negative_docs)):
    sentence = ' '.join(negative_docs[i])
    all_documents_string_array.append(sentence)
for i in range(len(positive_docs)):
    sentence = ' '.join(positive_docs[i])
    all_documents_string_array.append(sentence)

    
# Check the count of reviews.
len(all_documents_string_array)

2000

### Convert a collection of text documents to a matrix of token counts

It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

In [9]:

# We have kept 5000 as max feature value. which will consider top 5000 words from entire dataset.
# Consider the top max_features ordered by term frequency across the corpus.
vectorizer = CountVectorizer(max_features = 5000, binary=True)   
def vectorize_occurences(corpus, vectorizer):
    X = vectorizer.fit_transform(corpus)
    return X

  

### Transformation

We have used TfidfTransformer.

Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. This is a common term weighting scheme in information retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

In [10]:
# Transform a count matrix to a normalized tf or tf-idf representation
# tf(t, d) is the number of times a term occurs in the given document.
def calculate_frequencies(corpus):
    tf_transformer = TfidfTransformer(use_idf=False).fit(corpus)
    X = tf_transformer.transform(corpus)
    return X

# Create bag of words for all review words which we had cleaning in previous step
bag_of_words_occurences = vectorize_occurences(all_documents_string_array, vectorizer)
bag_of_words_frequencies = calculate_frequencies(bag_of_words_occurences)
# print(bag_of_words_frequencies) 

In [11]:
# Create list of class values for positive and negative review data.
#1000 0 for negatives and 1000 1 for positives

number_of_documents_for_each_class = int(len(negative_docs))
negative_labels = np.zeros((1,number_of_documents_for_each_class), dtype=int)[0]
number_of_documents_for_each_class = int(len(positive_docs))
positive_labels = np.ones((1,number_of_documents_for_each_class), dtype=int)[0]


# Assign bag of words for negative and positive 
negative_bow = bag_of_words_frequencies[:number_of_documents_for_each_class]
positive_bow = bag_of_words_frequencies[number_of_documents_for_each_class:]


print('Negative Bag of words')
print(negative_bow)

print('\nPositive Bag of words')
print(positive_bow)

Negative Bag of words
  (0, 38)	0.06696495301824251
  (0, 40)	0.06696495301824251
  (0, 42)	0.06696495301824251
  (0, 62)	0.06696495301824251
  (0, 83)	0.06696495301824251
  (0, 87)	0.06696495301824251
  (0, 130)	0.06696495301824251
  (0, 165)	0.06696495301824251
  (0, 169)	0.06696495301824251
  (0, 171)	0.06696495301824251
  (0, 181)	0.06696495301824251
  (0, 236)	0.06696495301824251
  (0, 240)	0.06696495301824251
  (0, 270)	0.06696495301824251
  (0, 297)	0.06696495301824251
  (0, 307)	0.06696495301824251
  (0, 316)	0.06696495301824251
  (0, 334)	0.06696495301824251
  (0, 345)	0.06696495301824251
  (0, 349)	0.06696495301824251
  (0, 402)	0.06696495301824251
  (0, 442)	0.06696495301824251
  (0, 453)	0.06696495301824251
  (0, 461)	0.06696495301824251
  (0, 511)	0.06696495301824251
  :	:
  (999, 4311)	0.07474350927519359
  (999, 4314)	0.07474350927519359
  (999, 4322)	0.07474350927519359
  (999, 4343)	0.07474350927519359
  (999, 4361)	0.07474350927519359
  (999, 4386)	0.07474350927519359

### Dataset details:
Data consist of 1000 positive reviews with class value = 1 and 1000 reviews of negative reviews which we have marked 0 as class value. 


### Multinomial Calssifier

In [12]:
#Create a Multinomial Naive Bayes Classifier
gnb = MultinomialNB()           #good
def multinomial_calssifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold):
    #Train the model using the training sets
    gnb.fit(features_training_set, labels_training_set)
    #Predict the response for test dataset
    pred = gnb.predict(features_test_set)
    # Model Accuracy, how often is the classifier correct?
    print("Multinomial Naive Bayes Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
    # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('multinomial_calssifier '+ str(fold)+'.xlsx')
    return average_accuracy

In [13]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data
    
    average_accuracy = multinomial_calssifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold)

print('---------- Completed Multinomial Calssifier --------')

Multinomial Naive Bayes Accuracy: 0.8475
Average accuracy:  0.8475

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       200
           1       0.89      0.79      0.84       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400


Confusion Matrix: 
[[181  19]
 [ 42 158]]

Multinomial Naive Bayes Accuracy: 0.83
Average accuracy:  0.83875

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       200
           1       0.87      0.78      0.82       200

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400


Confusion Matrix: 
[[177  23]
 [ 45 155]]

Multinomial Naive Bayes Accuracy: 0.82
Average accuracy:  0.8325

              precision    recall  f1-score   support

  

### Random Forest Classifier

In [14]:
def random_forest_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold):   #best
    clf = RandomForestClassifier(n_estimators = 1000)
    #rfe = RFE(estimator=clf, step=0.5)
    clf = clf.fit(features_training_set, labels_training_set)     #maybe use rfe instean of clf??
    pred = clf.predict(features_test_set)
    print("Random Forest Classifier Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
    # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('random_forest_classifier '+ str(fold)+'.xlsx')
    return average_accuracy

In [15]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):    # 
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data
    
    average_accuracy = random_forest_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold)
    
print('---------- Completed Random Forest Classifier --------')

Random Forest Classifier Accuracy: 0.79
Average accuracy:  0.79

              precision    recall  f1-score   support

           0       0.74      0.90      0.81       200
           1       0.87      0.69      0.77       200

    accuracy                           0.79       400
   macro avg       0.80      0.79      0.79       400
weighted avg       0.80      0.79      0.79       400


Confusion Matrix: 
[[179  21]
 [ 63 137]]

Random Forest Classifier Accuracy: 0.8
Average accuracy:  0.795

              precision    recall  f1-score   support

           0       0.76      0.88      0.81       200
           1       0.86      0.72      0.78       200

    accuracy                           0.80       400
   macro avg       0.81      0.80      0.80       400
weighted avg       0.81      0.80      0.80       400


Confusion Matrix: 
[[176  24]
 [ 56 144]]

Random Forest Classifier Accuracy: 0.7925
Average accuracy:  0.7941666666666668

              precision    recall  f1-score   s

### Gradient Boosting Classifier

In [16]:
def gradient_boosting_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold):  #asa si asa
    clf = GradientBoostingClassifier(n_estimators=100)
    clf = clf.fit(features_training_set, labels_training_set)
    pred = clf.predict(features_test_set)
    print("Gradient Boosting Classifier Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
    # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('gradient_boosting_classifier '+ str(fold)+'.xlsx')
    return average_accuracy

In [17]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data
    
    average_accuracy = gradient_boosting_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold)
    
    print('---------- Completed Gradient Boosting Classifier --------')

Gradient Boosting Classifier Accuracy: 0.7875
Average accuracy:  0.7875

              precision    recall  f1-score   support

           0       0.78      0.81      0.79       200
           1       0.80      0.77      0.78       200

    accuracy                           0.79       400
   macro avg       0.79      0.79      0.79       400
weighted avg       0.79      0.79      0.79       400


Confusion Matrix: 
[[161  39]
 [ 46 154]]

---------- Completed Gradient Boosting Classifier --------
Gradient Boosting Classifier Accuracy: 0.785
Average accuracy:  0.78625

              precision    recall  f1-score   support

           0       0.80      0.76      0.78       200
           1       0.77      0.81      0.79       200

    accuracy                           0.79       400
   macro avg       0.79      0.78      0.78       400
weighted avg       0.79      0.79      0.78       400


Confusion Matrix: 
[[151  49]
 [ 37 163]]

---------- Completed Gradient Boosting Classifier ---

### Logistic Regression Classifier

In [18]:
def logistic_regression_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold):
    clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
    clf = clf.fit(features_training_set, labels_training_set)
    pred = clf.predict(features_test_set)
    print("Logistic Regression Classifier Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
        # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('logistic_regression_classifier '+ str(fold)+'.xlsx')
    
    return average_accuracy

In [19]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data
    
    average_accuracy = logistic_regression_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold) 
    
print('---------- Completed Logistic Regression Classifier --------')

Logistic Regression Classifier Accuracy: 0.83
Average accuracy:  0.83

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       200
           1       0.86      0.79      0.82       200

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400


Confusion Matrix: 
[[175  25]
 [ 43 157]]

Logistic Regression Classifier Accuracy: 0.8425
Average accuracy:  0.8362499999999999

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       200
           1       0.84      0.84      0.84       200

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400


Confusion Matrix: 
[[169  31]
 [ 32 168]]

Logistic Regression Classifier Accuracy: 0.855
Average accuracy:  0.8424999999999999

              

### SVM Classifier

In [20]:
def svm_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold):  #best
    clf = svm.SVC(gamma='scale')
    clf = clf.fit(features_training_set, labels_training_set)
    pred = clf.predict(features_test_set)
    print("SVM Classifier Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
        # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('svm_classifier '+ str(fold)+'.xlsx')
    return average_accuracy

In [21]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data

    average_accuracy = svm_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold)
    
print('---------- Completed SVM Classifier --------')

SVM Classifier Accuracy: 0.8325
Average accuracy:  0.8325

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       200
           1       0.87      0.79      0.82       200

    accuracy                           0.83       400
   macro avg       0.84      0.83      0.83       400
weighted avg       0.84      0.83      0.83       400


Confusion Matrix: 
[[176  24]
 [ 43 157]]

SVM Classifier Accuracy: 0.8525
Average accuracy:  0.8425

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       200
           1       0.87      0.83      0.85       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400


Confusion Matrix: 
[[174  26]
 [ 33 167]]

SVM Classifier Accuracy: 0.8525
Average accuracy:  0.8458333333333333

              precision    recall  f1-score   support

           0  

### SVM Linear Classifier

In [22]:
def svm_linear_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy, fold): #good
    clf = svm.LinearSVC()
    clf = clf.fit(features_training_set, labels_training_set)
    pred = clf.predict(features_test_set)
    print("Linear SVM Classifier Accuracy:",metrics.accuracy_score(labels_test_set, pred))
    average_accuracy += metrics.accuracy_score(labels_test_set, pred)
    print("Average accuracy: ", average_accuracy/fold)
    print("")
    print(metrics.classification_report(labels_test_set, pred))
    print("")
    print("Confusion Matrix: ")
    print(metrics.confusion_matrix(labels_test_set, pred))
    print("")
        # Convert class label lists to dictionary 
    dict = {'Actual': labels_test_set, 'Predicted': pred} 
    df = pd.DataFrame(dict)
    # Save data to excel
    df.to_excel('svm_linear_classifier '+ str(fold)+'.xlsx')
    return average_accuracy

In [23]:
# Use cross validation for greater accuracy
kfold = KFold(n_splits = 5)  
fold = 0; average_accuracy = 0;

for train, test in kfold.split(negative_bow):
    fold = fold+1
    features_training_set = negative_bow[train].toarray().tolist()  # get training data
    features_training_set += positive_bow[train].toarray().tolist() # get training data

    features_test_set = negative_bow[test].toarray().tolist()       # get testing data
    features_test_set += positive_bow[test].toarray().tolist()      # get testing data
    
    labels_training_set = negative_labels[train].tolist()           # get training label(class) data
    labels_training_set += positive_labels[train].tolist()          # get training label(class) data
    
    labels_test_set = negative_labels[test].tolist()                # get testing label(class) data
    labels_test_set += positive_labels[test].tolist()               # get testing label(class) data
    
    average_accuracy = svm_linear_classifier(features_training_set, labels_training_set, features_test_set, labels_test_set, average_accuracy ,fold)

print('---------- Completed SVM Linear Classifier --------')

Linear SVM Classifier Accuracy: 0.835
Average accuracy:  0.835

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       200
           1       0.86      0.80      0.83       200

    accuracy                           0.83       400
   macro avg       0.84      0.83      0.83       400
weighted avg       0.84      0.83      0.83       400


Confusion Matrix: 
[[174  26]
 [ 40 160]]

Linear SVM Classifier Accuracy: 0.855
Average accuracy:  0.845

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       200
           1       0.86      0.85      0.85       200

    accuracy                           0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400


Confusion Matrix: 
[[172  28]
 [ 30 170]]

Linear SVM Classifier Accuracy: 0.8475
Average accuracy:  0.8458333333333333

              precision    recall  f1-score   suppor

### Output Details and Output File